In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
file_path ='/content/drive/MyDrive/Colab Notebooks/Bengali Dataset.csv'
data = pd.read_csv(file_path)

texts = data['Text'].tolist()
labels = data['Humor'].tolist()

label_dict = {label: idx for idx, label in enumerate(set(labels))}
numeric_labels = [label_dict[label] for label in labels]

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, numeric_labels, test_size=0.1, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_dict))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
epochs = 20
batch_size = 64
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

train_accuracy_values = []

for epoch in range(epochs):
    model.train()
    total_train_accuracy = 0
    total_batches = 0

    for i in range(0, len(train_encodings['input_ids']), batch_size):
        optimizer.zero_grad()
        batch_input = {key: val[i:i+batch_size].to(device) for key, val in train_encodings.items()}
        labels = train_labels[i:i+batch_size].to(device)
        outputs = model(**batch_input, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        predictions = torch.argmax(outputs.logits, dim=1)
        batch_accuracy = torch.sum(predictions == labels).item() / len(predictions)
        total_train_accuracy += batch_accuracy
        total_batches += 1

    epoch_train_accuracy = total_train_accuracy / total_batches
    train_accuracy_values.append(epoch_train_accuracy)
    print(f"Epoch {epoch+1} - Train Accuracy: {epoch_train_accuracy}")

# Test Evaluation
model.eval()
test_input = {key: val.to(device) for key, val in test_encodings.items()}
with torch.no_grad():
    outputs = model(**test_input)
    predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()

test_accuracy = accuracy_score(test_labels, predictions)
test_precision = precision_score(test_labels, predictions, average='weighted')
test_f1 = f1_score(test_labels, predictions, average='weighted')

print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test F1 Score: {test_f1}")


Epoch 1 - Train Accuracy: 0.4539930555555556
Epoch 2 - Train Accuracy: 0.4696180555555556
Epoch 3 - Train Accuracy: 0.5512152777777778
Epoch 4 - Train Accuracy: 0.556423611111111
Epoch 5 - Train Accuracy: 0.5833333333333334
Epoch 6 - Train Accuracy: 0.5225694444444444
Epoch 7 - Train Accuracy: 0.5303819444444444
Epoch 8 - Train Accuracy: 0.5694444444444444
Epoch 9 - Train Accuracy: 0.5616319444444444
Epoch 10 - Train Accuracy: 0.6276041666666666
Epoch 11 - Train Accuracy: 0.6458333333333334
Epoch 12 - Train Accuracy: 0.6588541666666666
Epoch 13 - Train Accuracy: 0.6432291666666666
Epoch 14 - Train Accuracy: 0.6458333333333334
Epoch 15 - Train Accuracy: 0.6640625
Epoch 16 - Train Accuracy: 0.6484375
Epoch 17 - Train Accuracy: 0.6536458333333334
Epoch 18 - Train Accuracy: 0.6484375
Epoch 19 - Train Accuracy: 0.6744791666666666
Epoch 20 - Train Accuracy: 0.6588541666666666
Test Accuracy: 0.65
Test Precision: 0.6699999999999999
Test F1 Score: 0.6446969696969698
